In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.metrics import mean_squared_error

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn import  linear_model
import tensorflow as tf
from tensorflow import keras

/kaggle/input/global-weather-repository/GlobalWeatherRepository.csv
/kaggle/input/global-weather-repository/state.db


In [2]:
df_train = pd.read_csv('../input/global-weather-repository/GlobalWeatherRepository.csv')
df_test = pd.read_csv('../input/global-weather-repository/GlobalWeatherRepository.csv')

In [3]:
df_train.columns
## Since there are too many columns, I will need to delete some that I think are unnecessary for the Model I'm training
## it will be easier to list the ones I'm keeping

Index(['country', 'location_name', 'latitude', 'longitude', 'timezone',
       'last_updated_epoch', 'last_updated', 'temperature_celsius',
       'temperature_fahrenheit', 'condition_text', 'wind_mph', 'wind_kph',
       'wind_degree', 'wind_direction', 'pressure_mb', 'pressure_in',
       'precip_mm', 'precip_in', 'humidity', 'cloud', 'feels_like_celsius',
       'feels_like_fahrenheit', 'visibility_km', 'visibility_miles',
       'uv_index', 'gust_mph', 'gust_kph', 'air_quality_Carbon_Monoxide',
       'air_quality_Ozone', 'air_quality_Nitrogen_dioxide',
       'air_quality_Sulphur_dioxide', 'air_quality_PM2.5', 'air_quality_PM10',
       'air_quality_us-epa-index', 'air_quality_gb-defra-index', 'sunrise',
       'sunset', 'moonrise', 'moonset', 'moon_phase', 'moon_illumination'],
      dtype='object')

In [4]:
df_train.describe()

,latitude,longitude,last_updated_epoch,temperature_celsius,temperature_fahrenheit,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
count,46967.000000,46967.000000,4.696700e+04,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,...,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000,46967.000000
mean,19.143160,22.210006,1.726313e+09,23.269102,73.885805,8.306815,13.372474,171.392978,1013.613899,29.931414,...,19.348006,511.616258,62.774014,13.678933,10.301289,23.021114,45.306193,1.631486,2.461409,48.643580
std,24.482211,65.783329,6.087832e+06,8.997838,16.196058,10.006371,16.102573,102.974994,6.997895,0.206537,...,17.766285,999.447249,38.032385,25.423819,52.329466,45.825141,137.362847,0.949271,2.429694,34.914705
min,-41.300000,-175.200000,1.715849e+09,-24.800000,-12.600000,2.200000,3.600000,1.000000,971.000000,28.670000,...,3.600000,-9999.000000,0.000000,0.000000,-9999.000000,0.185000,0.185000,1.000000,1.000000,0.000000
25%,3.750000,-6.250000,1.720963e+09,18.500000,65.300000,4.000000,6.500000,84.000000,1010.000000,29.830000,...,10.800000,217.000000,37.000000,0.800000,0.700000,4.600000,7.400000,1.000000,1.000000,14.000000
50%,17.250000,23.320000,1.726402e+09,25.400000,77.700000,6.900000,11.200000,165.000000,1013.000000,29.910000,...,17.000000,307.100000,59.000000,2.900000,2.035000,11.655000,18.300000,1.000000,1.000000,48.000000
75%,40.400000,50.580000,1.731576e+09,29.000000,84.200000,11.600000,18.700000,259.000000,1017.000000,30.030000,...,25.800000,468.050000,83.700000,13.690000,7.585000,25.700000,41.255000,2.000000,3.000000,83.000000
max,64.150000,179.220000,1.736768e+09,49.200000,120.600000,1841.200000,2963.200000,360.000000,1080.000000,31.890000,...,2970.400000,38879.398000,480.700000,427.700000,521.330000,1614.100000,5858.020000,6.000000,10.000000,100.000000


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46967 entries, 0 to 46966
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       46967 non-null  object 
 1   location_name                 46967 non-null  object 
 2   latitude                      46967 non-null  float64
 3   longitude                     46967 non-null  float64
 4   timezone                      46967 non-null  object 
 5   last_updated_epoch            46967 non-null  int64  
 6   last_updated                  46967 non-null  object 
 7   temperature_celsius           46967 non-null  float64
 8   temperature_fahrenheit        46967 non-null  float64
 9   condition_text                46967 non-null  object 
 10  wind_mph                      46967 non-null  float64
 11  wind_kph                      46967 non-null  float64
 12  wind_degree                   46967 non-null  int64  
 13  w

In [6]:
df_train.isnull().any()


country                         False
location_name                   False
latitude                        False
longitude                       False
timezone                        False
last_updated_epoch              False
last_updated                    False
temperature_celsius             False
temperature_fahrenheit          False
condition_text                  False
wind_mph                        False
wind_kph                        False
wind_degree                     False
wind_direction                  False
pressure_mb                     False
pressure_in                     False
precip_mm                       False
precip_in                       False
humidity                        False
cloud                           False
feels_like_celsius              False
feels_like_fahrenheit           False
visibility_km                   False
visibility_miles                False
uv_index                        False
gust_mph                        False
gust_kph    

In [7]:
df_train_num = df_train[list(df_train.dtypes[df_train.dtypes != 'object'].index)]
df_train_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46967 entries, 0 to 46966
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   latitude                      46967 non-null  float64
 1   longitude                     46967 non-null  float64
 2   last_updated_epoch            46967 non-null  int64  
 3   temperature_celsius           46967 non-null  float64
 4   temperature_fahrenheit        46967 non-null  float64
 5   wind_mph                      46967 non-null  float64
 6   wind_kph                      46967 non-null  float64
 7   wind_degree                   46967 non-null  int64  
 8   pressure_mb                   46967 non-null  float64
 9   pressure_in                   46967 non-null  float64
 10  precip_mm                     46967 non-null  float64
 11  precip_in                     46967 non-null  float64
 12  humidity                      46967 non-null  int64  
 13  c

In [8]:
## Here I will be listing the independent variables (X) and the target variable Temperature(F) (Y)
X = df_train_num[['wind_mph', 'humidity', 'cloud', 'pressure_mb', 'precip_in']]
y = df_train_num['temperature_fahrenheit']

In [9]:
X = df_train_num
y = df_train_num.pop('temperature_fahrenheit')

In [10]:
df_train_num = pd.DataFrame(X,y)
df_train_num

,latitude,longitude,last_updated_epoch,temperature_celsius,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
temperature_fahrenheit,,,,,,,,,,,,,,,,,,,,,
79.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 8114)
train_x.head()

,latitude,longitude,last_updated_epoch,temperature_celsius,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
15498,42.6800,23.3200,1722860100,28.4,9.4,15.1,28,1010.0,29.82,0.0,...,17.4,205.30,95.8,0.40,0.300,2.600,3.100,1,1,0
23274,36.7600,3.0500,1726317000,25.0,21.3,34.2,58,1022.0,30.18,0.0,...,39.3,299.70,90.0,5.18,3.145,7.400,23.125,1,1,78
24703,37.9800,23.7200,1726915500,24.2,11.6,18.7,13,1018.0,30.06,0.0,...,21.5,347.80,62.0,15.17,2.960,10.175,13.320,1,1,88
38929,42.4411,19.2636,1733222700,11.1,2.2,3.6,202,1017.0,30.03,0.0,...,10.8,431.05,48.0,19.61,7.215,29.230,30.155,2,3,3
18967,39.9300,32.8600,1724328000,33.2,5.6,9.0,50,1007.0,29.74,0.0,...,16.2,212.00,124.5,3.30,2.600,6.500,8.000,1,1,93


In [12]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [13]:
prediction1 = regr.predict(test_x)
print(prediction1)

[ 91.94422287  53.5985235   91.2183165  ...  80.96359103  57.92509825
 103.46003677]


In [14]:
pd.DataFrame({'actual': test_y, 'prediction': prediction2, 'difference':(test_y - prediction1)})

NameError: name 'prediction2' is not defined

In [ ]:
## Decision Tree Model
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 8114)
regressor.fit(X,y)

In [ ]:
prediction2 = regressor.predict(test_x)
print(prediction2)

In [ ]:
pd.DataFrame({'actual': test_y, 'prediction': prediction2, 'difference':(test_y - prediction2)})